In [1]:
#hide earrings 
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC #Import svm model
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline, make_pipeline

In [4]:
df=pd.read_csv(r'C:\Users\Admin\python\car_data.csv')  #Can be any csv of your choice.

In [5]:
df.head(2)

,User ID,Gender,Age,AnnualSalary,Purchased
0,385,Male,35,20000,0
1,681,Male,40,43500,0


In [6]:
df.isnull().sum()

User ID         0
Gender          0
Age             0
AnnualSalary    0
Purchased       0
dtype: int64

In [7]:
df.shape

(1000, 5)

#### plain or steps for pipeline

In [8]:
#Separate Feature and Target Matrix
def get_seprate_features(df_id,col_id):
    #col_id is special col or target col b/c this the whole model is going to be created 
    global X, y
    X = df_id.drop(col_id,axis = 1) 
    y = df_id[col_id]
get_seprate_features(df,'Purchased')

In [9]:
X.head(2)

,User ID,Gender,Age,AnnualSalary
0,385,Male,35,20000
1,681,Male,40,43500


In [10]:
y.head(2)

0    0
1    0
Name: Purchased, dtype: int64

In [11]:
def get_train_test_split(array1, array2, test_size, random_state):
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(array1, array2, test_size=test_size,random_state=random_state)
get_train_test_split(X,y,0.2,42)

In [12]:
X_train.head(2)

,User ID,Gender,Age,AnnualSalary
29,994,Female,38,147500
535,833,Female,59,24500


In [13]:
y_train.sample(2)

962    0
531    0
Name: Purchased, dtype: int64

In [14]:
def get_trf1(col_index_id):
    global trf1
    trf1 = ColumnTransformer(
    [('ohe_gender',OneHotEncoder(sparse=False, handle_unknown='ignore'),[col_index_id])
    ],remainder = "passthrough")
get_trf1(1)

In [15]:
trf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_gender',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1])])

In [16]:
def get_trf2(slice_1st,slice_last):
    global trf2
    trf2 = ColumnTransformer(
    [('scale',MinMaxScaler(),slice(slice_1st,slice_last))
    ])
get_trf2(0,4)

In [17]:
trf2

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 4, None))])

In [18]:
def get_trf3(k_id):
    global trf3
    # 4) Feature Sesection (select (the k) means most important columns)
    trf3 = SelectKBest(score_func=chi2,k=k_id)
get_trf3(3)

In [19]:
trf3

SelectKBest(k=3, score_func=<function chi2 at 0x000001FD90D73700>)

In [20]:
def get_trf4(mod_id):
    global trf4
    # 5) Train model (by a particular model)
    trf4 = mod_id
get_trf4(SVC())

In [21]:
trf4

SVC()

In [22]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)])

In [23]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_gender',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 4, None))])),
                ('trf3',
                 SelectKBest(k=3,
                             score_func=<function chi2 at 0x000001FD90D73700>)),
                ('trf4', SVC())])

In [28]:
pipe.named_steps['trf1'].transformers_[0][1].categories_

[array(['Female', 'Male'], dtype=object)]

In [29]:
pipe.named_steps['trf2'].transformers_[0][1].scale_

array([1.        , 1.        , 0.00100402, 0.02222222])

In [30]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_gender',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 4, None))])),
                ('trf3',
                 SelectKBest(k=3,
                             score_func=<function chi2 at 0x000001FD90D73700>)),
                ('trf4', SVC())])

In [31]:
from sklearn import set_config
set_config(display='diagram')

In [32]:
y_pred = pipe.predict(X_test)

In [33]:
accuracy_score(y_test,y_pred)

0.85

In [34]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()
# cv ==bcross_val_score
# you can take cross_val_score by using pipe,
# it train and split dataset at 5 different times and every time run the algorithm, ccalculate the mean score of accuracy 

0.8125

In [35]:
import pickle
#here we doing encoding
pickle.dump(pipe,open('pipesvc.pkl','wb'))
#these are binary file you can not see however you can use it 